In [ ]:
!git clone https://github.com/pt1127/Streaming-Data-With-Spark.git

In [ ]:
# Install PySpark and Spark NLP
! pip install -q pyspark==3.1.2 spark-nlp

In [11]:
from pyspark.ml import PipelineModel

In [ ]:
best_model = PipelineModel.load('/content/Streaming-Data-With-Spark/best_model')

In [ ]:
example = spark.createDataFrame([['Tôi là một con người tốt bụng']]).toDF("clean_text")
result = best_model.transform(example).select('prediction')